In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import time

# Read in data

In [65]:
bz = pd.read_csv('yelp_dataset/restaurant.csv',index_col='Unnamed: 0')
rv = pd.read_csv('yelp_dataset/review_restaurant.csv',index_col='Unnamed: 0')
us = pd.read_csv('yelp_dataset/user.csv',index_col='Unnamed: 0')

# bz --> rv --> us

In [4]:
# bz1 = bz.sample(frac=0.1)
# rv1 = bz1[['business_id']].merge(rv, how='left',on='business_id')
# us1 = rv1[['user_id']].merge(rv1, how='left',on='user_id')

In [5]:
df = rv.groupby(['business_id','user_id']).agg({'stars':'mean'}).reset_index()

In [7]:
df.head()

,business_id,user_id,stars
0,--9e1ONYQuAa-CB_Rrw7Tw,---1lKK3aKOuomHnwAkAow,4.0
1,--9e1ONYQuAa-CB_Rrw7Tw,-11nMat3F0DgsxT30hn8zA,5.0
2,--9e1ONYQuAa-CB_Rrw7Tw,-3fMaL_ck0wzEsTZyz-mqA,5.0
3,--9e1ONYQuAa-CB_Rrw7Tw,-47EprTYhe9IbAvplGIcpA,5.0
4,--9e1ONYQuAa-CB_Rrw7Tw,-7cPERHtAupd1Xq53nmhXg,5.0


In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['business_id'] = le.fit_transform(df['business_id'].values)
df['user_id'] = le.fit_transform(df['user_id'].values)

# MF Attempt

In [15]:
msk = np.random.rand(len(df)) < 0.7
df_train = df[msk]
user_ind = [x for x in df_train['user_id'].index]
item_ind = [x for x in df_train['business_id'].index]
stars = df_train['stars'].values

In [ ]:
df_train['user_id'].nunique()

In [ ]:
df_train['business_id'].nunique()

In [8]:
# variables
feature_len = 10
U = tf.Variable(initial_value=tf.truncated_normal([373483,feature_len]), name='users')
P = tf.Variable(initial_value=tf.truncated_normal([feature_len,8848]), name='items')
result = tf.matmul(U, P)
result_flatten = tf.reshape(result, [-1])

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# with tf.Session() as s: x = s.run(tf.shape(result)[1])
# type(x)
# type(user_ind)
# np.multiply(user_ind,x)

In [ ]:
# s = tf.Session()
# np.multiply(user_ind, s.run(tf.shape(result)[1]))

In [ ]:
# s.run(tf.shape(result)[1])

In [ ]:
# # with tf.Session() as s: x = s.run(tf.shape(result)[1])
# R = tf.gather(result_flatten, np.multiply(user_ind, tf.shape(result)[1]) + 
#               item_ind, name='extracting_user_stars')
# np.add(np.multiply(user_ind, x),item_ind)

In [9]:
with tf.Session() as s: x = s.run(tf.shape(result)[1])

In [17]:
y = np.add(np.multiply(user_ind, x),item_ind)

In [18]:
y

array([          0,        8849,       17698, ..., 11899621958,
       11899630807, 11899639656])

In [19]:
R = tf.gather(result_flatten, y, name='extracting_user_stars')

In [ ]:
# with tf.Session() as s: x = s.run(tf.shape(result)[1])
# R = tf.gather(result_flatten, np.add(np.multiply(user_ind, x),item_ind), name='extracting_user_stars')

In [24]:
with tf.Session() as s: s.run(R)

FailedPreconditionError: Attempting to use uninitialized value users
	 [[node users/read (defined at <ipython-input-8-0c956467293a>:3) ]]

Caused by op 'users/read', defined at:
  File "/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/anaconda3/lib/python3.7/asyncio/base_events.py", line 539, in run_forever
    self._run_once()
  File "/anaconda3/lib/py

In [20]:
# cost function
diff_op = tf.subtract(R, stars, name='training_diff')
diff_op_squared = tf.abs(diff_op, name="squared_difference")
base_cost = tf.reduce_sum(diff_op_squared, name="sum_squared_error")

In [21]:
# regularization
lda = tf.constant(.001, name='lambda')
norm_sums = tf.add(tf.reduce_sum(tf.abs(U, name='user_abs'), name='user_norm'), 
   tf.reduce_sum(tf.abs(P, name='item_abs'), name='item_norm'))
regularizer = tf.multiply(norm_sums, lda, 'regularizer')

cost = tf.add(base_cost, regularizer)

In [22]:
# optimizer
lr = tf.constant(.001, name='learning_rate')
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.96, staircase=True)

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(cost, global_step=global_step)

Instructions for updating:
Use tf.cast instead.


ValueError: Tried to convert 'tensor' to a tensor and failed. Error: Dimension size, given by scalar input 2, must be non-negative but is -990389712 for 'gradients/Reshape_grad/Reshape/UnsortedSegmentSum' (op: 'UnsortedSegmentSum') with input shapes: [941491], [941491], [] and with computed input tensors: input[2] = <-990389712>.

In [ ]:
# execution
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)
for i in xrange(500):
    sess.run(training_step)


In [ ]:
# testing
u, p, r = df[['user', 'item', 'rate']].values[0]
rhat = tf.gather(tf.gather(result, u-1), p-1)
print "rating for user " + str(u) + " for item " + str(p) + 
      " is " + str(r) + " and our prediction is: " + str(sess.run(rhat))

In [ ]:
# calculate accuracy
df_test = df[~msk]
user_indecies_test = [x-1 for x in df_test.user.values]
item_indecies_test = [x-1 for x in df_test.item.values]
rates_test = df_test.rate.values

# accuracy
R_test = tf.gather(result_flatten, user_indecies_test * tf.shape(result)[1] + item_indecies_test, name='extracting_user_rate_test')
diff_op_test = tf.sub(R_test, rates_test, name='test_diff')
diff_op_squared_test = tf.abs(diff_op, name="squared_difference_test")

cost_test = tf.div(tf.reduce_sum(tf.square(diff_op_squared_test, name="squared_difference_test"), name="sum_squared_error_test"), df_test.shape[0] * 2, name="average_error")
print sess.run(cost_test)

# CF Attempt

In [ ]:
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV

from surprise import SVD
from surprise import Dataset
from surprise import NormalPredictor
from surprise import Reader
from surprise import accuracy
from surprise import similarities
from surprise import KNNBasic

In [ ]:
# bz1 = bz.sample(frac=0.1)
# rv1 = bz1[['business_id']].merge(rv, how='left',on='business_id')
# us1 = rv1[['user_id']].merge(rv1, how='left',on='user_id')

In [ ]:
# reader, the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['user_id', 'business_id', 'stars']], reader)

In [ ]:
cross_validate(NormalPredictor(), data, cv=5)

In [ ]:
# split the data into test and training
trainset, testset = train_test_split(data, test_size=0.2)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

In [ ]:
cross_validate(algo, data, measures=['rmse', 'mae'], cv=5,
               return_train_measures=False, n_jobs=1,
               pre_dispatch='2*n_jobs', verbose=False)

In [ ]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

#### cosine similarity

In [ ]:
sim_options = {'name': 'cosine'}
algo2 = KNNBasic(sim_options=sim_options)

algo2.fit(trainset)
similarities = algo2.test(testset)

accuracy.rmse(similarities)

In [ ]:
sim_options2 = {'name': 'pearson'}
algo3 = KNNBasic(sim_options=sim_options)

algo3.fit(trainset)
similarities = algo2.test(testset)

accuracy.rmse(similarities)

In [ ]:
cross_validate(algo2, data, measures=['rmse', 'mae'], cv=2,
               return_train_measures=False, n_jobs=1,
               pre_dispatch='2*n_jobs', verbose=False)

# CF Attempt 2

In [ ]:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]

In [ ]:
data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [ ]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [ ]:
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

# MF Attempt 2

Code from https://blog.insightdatascience.com/explicit-matrix-factorization-als-sgd-and-all-that-jazz-b00e4d9b21ea

In [61]:
df.to_numpy()

array([['--9e1ONYQuAa-CB_Rrw7Tw', '---1lKK3aKOuomHnwAkAow', 4.0],
       ['--9e1ONYQuAa-CB_Rrw7Tw', '-11nMat3F0DgsxT30hn8zA', 5.0],
       ['--9e1ONYQuAa-CB_Rrw7Tw', '-3fMaL_ck0wzEsTZyz-mqA', 5.0],
       ...,
       ['zzzaIBwimxVej4tY6qFOUQ', 'spceyljVwAJ8vKjlI5wllQ', 3.0],
       ['zzzaIBwimxVej4tY6qFOUQ', 'sxEjivCOzY3tVcvVWA0EvQ', 5.0],
       ['zzzaIBwimxVej4tY6qFOUQ', 'vwLV4TkRMoghs86tB1Ke8Q', 3.0]],
      dtype=object)

In [2]:
n_users = df.user_id.unique().shape[0]
n_items = df.business_id.unique().shape[0]
ratings = np.zeros((n_users, n_items))

for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
ratings

NameError: name 'df' is not defined

In [41]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

In [ ]:
train, test = train_test_split(ratings)

In [31]:
from numpy.linalg import solve

class ExplicitMF():
    def __init__(self, 
                 ratings, 
                 n_factors=40, 
                 item_reg=0.0, 
                 user_reg=0.0,
                 verbose=False):
        """
        Train a matrix factorization model to predict empty 
        entries in a matrix. The terminology assumes a 
        ratings matrix which is ~ user x item
        
        Params
        ======
        ratings : (ndarray)
            User x Item matrix with corresponding ratings
        
        n_factors : (int)
            Number of latent factors to use in matrix 
            factorization model
        
        item_reg : (float)
            Regularization term for item latent factors
        
        user_reg : (float)
            Regularization term for user latent factors
        
        verbose : (bool)
            Whether or not to printout training progress
        """
        
        self.ratings = ratings
        self.n_users, self.n_items = ratings.shape
        self.n_factors = n_factors
        self.item_reg = item_reg
        self.user_reg = user_reg
        self._v = verbose

    def als_step(self,
                 latent_vectors,
                 fixed_vecs,
                 ratings,
                 _lambda,
                 type='user'):
        """
        One of the two ALS steps. Solve for the latent vectors
        specified by type.
        """
        if type == 'user':
            # Precompute
            YTY = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(YTY.shape[0]) * _lambda

            for u in range(latent_vectors.shape[0]):
                latent_vectors[u, :] = solve((YTY + lambdaI), 
                                             ratings[u, :].dot(fixed_vecs))
        elif type == 'item':
            # Precompute
            XTX = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(XTX.shape[0]) * _lambda
            
            for i in range(latent_vectors.shape[0]):
                latent_vectors[i, :] = solve((XTX + lambdaI), 
                                             ratings[:, i].T.dot(fixed_vecs))
        return latent_vectors

    def train(self, n_iter=10):
        """ Train model for n_iter iterations from scratch."""
        # initialize latent vectors
        self.user_vecs = np.random.random((self.n_users, self.n_factors))
        self.item_vecs = np.random.random((self.n_items, self.n_factors))
        
        self.partial_train(n_iter)
    
    def partial_train(self, n_iter):
        """ 
        Train model for n_iter iterations. Can be 
        called multiple times for further training.
        """
        ctr = 1
        while ctr <= n_iter:
            if ctr % 10 == 0 and self._v:
                print('\tcurrent iteration: {}'.format(ctr))
            self.user_vecs = self.als_step(self.user_vecs, 
                                           self.item_vecs, 
                                           self.ratings, 
                                           self.user_reg, 
                                           type='user')
            self.item_vecs = self.als_step(self.item_vecs, 
                                           self.user_vecs, 
                                           self.ratings, 
                                           self.item_reg, 
                                           type='item')
            ctr += 1
    
    def predict_all(self):
        """ Predict ratings for every user and item. """
        predictions = np.zeros((self.user_vecs.shape[0], 
                                self.item_vecs.shape[0]))
        for u in xrange(self.user_vecs.shape[0]):
            for i in xrange(self.item_vecs.shape[0]):
                predictions[u, i] = self.predict(u, i)
                
        return predictions
    def predict(self, u, i):
        """ Single user and item prediction. """
        return self.user_vecs[u, :].dot(self.item_vecs[i, :].T)
    
    def calculate_learning_curve(self, iter_array, test):
        """
        Keep track of MSE as a function of training iterations.
        
        Params
        ======
        iter_array : (list)
            List of numbers of iterations to train for each step of 
            the learning curve. e.g. [1, 5, 10, 20]
        test : (2D ndarray)
            Testing dataset (assumed to be user x item).
        
        The function creates two new class attributes:
        
        train_mse : (list)
            Training data MSE values for each value of iter_array
        test_mse : (list)
            Test data MSE values for each value of iter_array
        """
        iter_array.sort()
        self.train_mse =[]
        self.test_mse = []
        iter_diff = 0
        for (i, n_iter) in enumerate(iter_array):
            if self._v:
                print('Iteration: {}'.format(n_iter))
            if i == 0:
                self.train(n_iter - iter_diff)
            else:
                self.partial_train(n_iter - iter_diff)

            predictions = self.predict_all()

            self.train_mse += [get_mse(predictions, self.ratings)]
            self.test_mse += [get_mse(predictions, test)]
            if self._v:
                print('Train mse: ' + str(self.train_mse[-1]))
                print('Test mse: ' + str(self.test_mse[-1]))
            iter_diff = n_iter

In [32]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [ ]:
MF_ALS = ExplicitMF(train, n_factors=40, user_reg=0.0, item_reg=0.0)
iter_array = [1, 2, 5, 10, 25, 50, 100]
MF_ALS.calculate_learning_curve(iter_array, test)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

def plot_learning_curve(iter_array, model):
    plt.plot(iter_array, model.train_mse, \
             label='Training', linewidth=5)
    plt.plot(iter_array, model.test_mse, \
             label='Test', linewidth=5)


    plt.xticks(fontsize=16);
    plt.yticks(fontsize=16);
    plt.xlabel('iterations', fontsize=30);
    plt.ylabel('MSE', fontsize=30);
    plt.legend(loc='best', fontsize=20);

In [ ]:
plot_learning_curve(iter_array, MF_ALS)

# Demo Check

In [66]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [67]:
demo = ['9ks-80ZxeCZWNObzEbW4-Q','pabMYegF28KjHQ5hybAJ0A','F_N4NyoMJWzBYFspeYfH3A',
        'KFWOn7x19RmrO6kzTJYmrQ','yi_4YTVS6XL7s5atKPbmpA','owbC7FP8SNAlwv6f9S5Stw',
        'AoII0-uM9z0EesGnb9YCjQ','lYyWUsl47zaEoKJQxuqLcQ','sGIGOFKg_yTd76cDPs6pjA']

In [179]:
demo_rv = rv[rv['user_id'].isin(demo)]
demo_us = us[us['user_id'].isin(demo)]
demo_bz = bz[bz['business_id'].isin(demo_rv['business_id'].unique().tolist())]

In [180]:
demo_rv.drop(['useful','funny','cool','review_id'],axis=1,inplace=True)
demo_rv = demo_rv.rename(columns={"stars": "rv.stars"})

In [171]:
demo_rv.head()

,business_id,user_id,rv.stars,text,date
4,PZ-LZzSlhSe9utkQYU8pFg,pabMYegF28KjHQ5hybAJ0A,3.0,"This is old Vegas, this atmosphere is old scho...",2009-06-01 20:02:55
5,PZ-LZzSlhSe9utkQYU8pFg,F_N4NyoMJWzBYFspeYfH3A,4.0,"Heavy on the atmosphere, heavy on the red sauc...",2010-05-22 16:54:57
19,PZ-LZzSlhSe9utkQYU8pFg,KFWOn7x19RmrO6kzTJYmrQ,5.0,Can't say much more than what everyone else ha...,2011-02-27 16:01:22
20,PZ-LZzSlhSe9utkQYU8pFg,F_N4NyoMJWzBYFspeYfH3A,4.0,When I heard the Liberace Museum was shutting ...,2010-12-17 20:30:54
23,PZ-LZzSlhSe9utkQYU8pFg,yi_4YTVS6XL7s5atKPbmpA,1.0,The Olive Garden by any other name is still Th...,2009-09-06 05:11:50


In [181]:
# to use as a merge
demo_us_merge = demo_us[['user_id','name']]

In [182]:
# to see users separately
compliments = demo_us.filter(regex='compliment').sum(axis=1)
demo_us1 = demo_us[demo_us.columns.drop(list(demo_us1.filter(regex='compliment')))]
demo_us1 = demo_us1.drop('friends',axis=1)
demo_us1['compliments'] = compliments
demo_us1 = demo_us1.rename(columns={"review_count": "us.review_count"})

KeyError: "['compliments'] not found in axis"

In [176]:
demo_us1

,user_id,name,us.review_count,yelping_since,useful,funny,cool,elite,fans,average_stars,compliments
4,pabMYegF28KjHQ5hybAJ0A,Steve,59,2009-03-03 16:53:04,45,10,12,NaN,2,3.80,7
5,F_N4NyoMJWzBYFspeYfH3A,Mike,401,2009-12-11 18:58:50,436,118,107,NaN,9,3.60,25
19,KFWOn7x19RmrO6kzTJYmrQ,Melanie,130,2010-07-12 03:56:38,390,139,195,"2012,2013",15,4.01,100
22,yi_4YTVS6XL7s5atKPbmpA,Joseph,167,2008-04-20 05:16:25,655,480,321,NaN,17,3.19,112
29,owbC7FP8SNAlwv6f9S5Stw,Girl,245,2012-09-28 19:36:16,484,259,177,NaN,8,3.44,25
35,AoII0-uM9z0EesGnb9YCjQ,Nick,57,2009-06-19 23:37:27,173,132,94,2011,3,3.83,33
75,lYyWUsl47zaEoKJQxuqLcQ,Michelle,662,2011-05-29 17:24:38,2166,1051,1305,"2012,2013,2014,2015,2016,2017",187,3.95,556
80,sGIGOFKg_yTd76cDPs6pjA,Krystal,100,2013-04-21 20:35:55,266,60,174,"2016,2017,2018",15,4.36,29
5231,9ks-80ZxeCZWNObzEbW4-Q,Cynthia,535,2012-09-21 03:40:08,2969,1732,2032,"2013,2014,2015,2016,2017,2018",102,3.84,931


In [185]:
demo_bz = demo_bz[demo_bz.columns.drop(list(demo_bz.filter(regex='attributes')))]
demo_bz = demo_bz.drop(['address','city','state','postal_code','latitude','longitude'], axis=1)
demo_bz = demo_bz.rename(columns={"name": "rt.name","stars": "rt.stars",
                        "review_count":"rt.review_count"})

In [154]:
demo_bz.head()

,business_id,rt.name,rt.stars,rt.review_count,is_open,categories
17,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,4.0,40,0,Restaurants
22,KWywu2tTEPWmR9JnBc0WyQ,Hunk Mansion,4.0,107,1,Bars
464,LEOvXK3lMEfjnex3CPvrng,Albinas Italian American Bakery,4.5,11,0,"Food,Bakeries,Restaurants"
1744,GMrwDXRlAZU2zj5nH6l4vQ,Pho Thanh Huong,4.0,481,1,"Restaurants,Sandwiches"
1975,ujHiaprwCQ5ewziu0Vi9rw,The Buffet at Bellagio,3.5,4227,1,"Buffets,Restaurants"


In [186]:
full_demo = demo_rv.merge(demo_us_merge, on='user_id', how='inner')
full_demo = full_demo.merge(demo_bz, on='business_id', how='inner')
# full_demo = demo_rv.merge(demo_bz, on='business_id', how='inner')

In [192]:
demo_list = []
for i in demo:
    demo_list.append(full_demo[full_demo['user_id']==i].drop(['business_id','user_id','text'],axis=1))

In [195]:
for i in ['rv.stars','rt.stars','is_open','categories']:
    print(demo_list[0][i].value_counts())

4.0    94
5.0    68
3.0    38
2.0    24
1.0     9
Name: rv.stars, dtype: int64
4.0    90
3.5    59
4.5    41
3.0    28
2.5    12
5.0     2
2.0     1
Name: rt.stars, dtype: int64
1    189
0     44
Name: is_open, dtype: int64
Restaurants                                                                                                                              33
Restaurants,Bars                                                                                                                          5
Restaurants,Burgers                                                                                                                       5
Bars                                                                                                                                      4
Coffee & Tea,Food                                                                                                                         4
Buffets,Restaurants                                                         

In [194]:
demo_list[0]['rv.stars'].value_counts()

,rv.stars,date,name,rt.name,rt.stars,rt.review_count,is_open,categories
10,5.0,2014-03-11 21:38:03,Cynthia,The Bootlegger Italian Bistro,4.0,969,1,"Restaurants,Steakhouses"
12,2.0,2013-11-14 05:07:05,Cynthia,Blue Martini Lounge,3.5,930,1,"Bars,Restaurants"
18,4.0,2014-12-20 22:31:03,Cynthia,The Shops at Crystals,3.5,346,1,Restaurants
57,5.0,2014-10-05 00:52:08,Cynthia,Golden Steer Steakhouse,4.0,873,1,"Restaurants,Seafood,Steakhouses,Salad"
64,4.0,2014-07-06 04:03:07,Cynthia,Nove Italiano,4.0,495,0,Restaurants
94,3.0,2016-06-24 21:35:58,Cynthia,Holsteins,4.0,3159,1,"Bars,Burgers,Restaurants"
168,5.0,2017-07-10 17:32:24,Cynthia,Il Mulino New York,4.0,305,1,Restaurants
177,5.0,2013-12-09 07:40:48,Cynthia,Secret Pizza,4.0,4286,1,"Pizza,Restaurants"
190,5.0,2015-01-02 03:54:25,Cynthia,Cucina by Wolfgang Puck,4.0,921,1,"Restaurants,Pizza"
201,4.0,2015-02-18 00:14:05,Cynthia,Pot Liquor CAS,3.5,832,0,"Barbeque,Restaurants,Soul Food"
